In [ ]:
from nets.RNA_graph_classification.load_net import gnn_model # import all GNN from data.data 
import LoadData # import dataset
%matplotlib inline
import dgl
import torch
import matplotlib.pyplot as plt import networkx as nx
from dgl.nn.pytorch import GraphConv

In [ ]:
#
import json

arg_config = "./configs/RNAgraph_graph_classification_GCN_in_vivo_100k.json"

with open(arg_config) as f:
    config = json.load(f)

In [ ]:
DATASET_NAME = "ALKBH5_Baltz2012"

In [ ]:
# Data load
dataset = LoadData(DATASET_NAME, config)

In [ ]:
dataset.train[0]

In [ ]:
def collate(samples):
    # The input `samples` is a list of pairs # (graph, label).
    graphs, labels = map(list, zip(*samples))
    batched_graph = dgl.batch(graphs)
    return batched_graph, torch.tensor(labels)

In [ ]:
# Model
import torch.nn as nn
import torch.nn.functional as F
class Classifier(nn.Module):
    def __init__(self, in_dim, hidden_dim, n_classes): super(Classifier, self).__init__()
        self.conv1 = GraphConv(in_dim, hidden_dim) 
        self.conv2 = GraphConv(hidden_dim, hidden_dim) 
        self.classify = nn.Linear(hidden_dim, n_classes)
        
    def forward(self, g):
        # Use node degree as the initial node feature. For undirected graphs, # is the same as the out_degree.
        h = g.in_degrees().view(-1, 1).float()
        # Perform graph convolution and activation function.
        h = F.relu(self.conv1(g, h))
        h = F.relu(self.conv2(g, h))
        g.ndata['h'] = h
        # Calculate graph representation by averaging all the node representa hg = dgl.mean_nodes(g, 'h')
        return self.classify(hg)

In [ ]:
# Data load
import torch.optim as optim
from torch.utils.data import DataLoader

# Create training and test sets.
trainset = dataset.train
testset = dataset.test

# Use PyTorch's DataLoader and the collate function
# defined before.
data_loader = DataLoader(trainset, batch_size=32, shuffle=True,
collate_fn=collate)

In [ ]:
# Create model
model = Classifier(1, 256, 2)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) model.train()

# Model
epoch_losses = []
for epoch in range(10):
    epoch_loss = 0
    for iter, (bg, label) in enumerate(data_loader):
        prediction = model(bg)
        loss = loss_func(prediction, label) optimizer.zero_grad() 
        loss.backward()
        optimizer.step()
        epoch_loss += loss.detach().item()
        epoch_loss /= (iter + 1)
        print('Epoch {}, loss {:.4f}'.format(epoch, epoch_loss)) 
        epoch_losses.append(epoch_loss)

In [ ]:
plt.title('cross entropy averaged over minibatches') 
plt.plot(epoch_losses)
plt.show()